<a href="https://colab.research.google.com/github/amiholz/NN/blob/master/Ex1_q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import Model

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)


test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv = Conv2D(2, 5, activation='relu')
        self.flatten = Flatten()
        self.d = Dense(10, activation='softmax')
    
    def call(self, x):
        x = self.conv(x)
        x = self.flatten(x)
        return self.d(x)



In [0]:
# Create an instance of the model
model = MyModel()


loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()


train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(images, labels):
    
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)



In [14]:
EPOCHS = 5
train_iter_num = 0
TRAIN_ITERS_FOR_PRINT = 500

train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []
num_of_iters = []

inside_epoch_template = 'Inside Epoch {}, Total Iters: {}, Train Loss: {}, Train Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
end_epoch_template = 'Ending Epoch {}, Total Iters: {},  Train Loss: {}, Train Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)
        train_iter_num += 1
        if train_iter_num % TRAIN_ITERS_FOR_PRINT == 0:
              
              for (test_images, test_labels) in test_ds:
                    test_step(test_images, test_labels)
              
              # train_losses.append(train_loss.result())
              # train_accuracies.append(train_accuracy.result()*100)
              # test_losses.append(test_loss.result())
              # test_accuracies.append(test_accuracy.result()*100))
              # num_of_iters.append(train_iter_num)
              
              #print(inside_epoch_template.format(epoch+1,train_iter_num,
                              # train_loss.result(),
                              # train_accuracy.result()*100,
                              # test_loss.result(),
                              # test_accuracy.result()*100))
    
    for (test_images, test_labels) in test_ds:
        test_step(test_images, test_labels)
           
    print(end_epoch_template.format(epoch+1,
                          train_iter_num,
                              train_loss.result(),
                              train_accuracy.result()*100,
                              test_loss.result(),
                              test_accuracy.result()*100))
    # test_accuracies.append()
    print(" -- ", flush=True)
    
    # Reset the metrics for the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Ending Epoch 1, Total Iters: 1875,  Train Loss: 0.33325982093811035, Train Accuracy: 90.49333190917969, Test Loss: 0.23699292540550232, Test Accuracy: 93.25
 -- 
Ending Epoch 2, Total Iters: 3750,  Train Loss: 0.1757410615682602, Train Accuracy: 94.961669921875, Test Loss: 0.1622951626777649, Test Accuracy: 95.33000183105469
 -- 
Ending Epoch 3, Total Iters: 5625,  Train Loss: 0.14308826625347137, Train Accuracy: 95.96833801269531, Test Loss: 0.1331891119480133, Test Accuracy: 96.10400390625
 -- 
Ending